In [46]:
# Imports

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from PIL import Image
import torchvision.models as models
from pathlib import Path
import pandas as pd
from padfunc import CustomResizeAndReflectPad
import matplotlib.pyplot as plt
import os
from torch import nn
import numpy as np
import json
import skimage
from skimage import measure, io, filters

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"


In [47]:
# Transformation to be applied to the images
simple_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    CustomResizeAndReflectPad(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [48]:
# Load the model
def load_model(MODEL_PATH, class_names):
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1).to(device)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(p=0.2),
        nn.Linear(num_ftrs, len(class_names))
    ).to(device)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    model.eval()
    return model

# Predict function
def predict_image(image_path, model, transform, class_names):
    image = Image.open(image_path)
    transformed_image = transform(image).unsqueeze(0).to(device)
    
    model.eval()
    with torch.inference_mode():
        pred_logit = model(transformed_image)
        predicted_prob = torch.softmax(pred_logit, dim=1)
        pred_label = torch.argmax(predicted_prob, dim=1).item()
        predicted_class = class_names[pred_label]
        predicted_prob = predicted_prob[0, pred_label].item()
        return predicted_class, predicted_prob

# Measure blob sizes
def measure_blob_sizes(image_path):
    """Measure sizes of all blobs in the image."""
    image = io.imread(image_path, as_gray=True)
    threshold = filters.threshold_otsu(image)
    binary = image > threshold
    labeled = measure.label(binary)
    properties = measure.regionprops(labeled)
    sizes = [prop.area for prop in properties]
    return sizes


# # Predict on all images in a folder
# def predict_folder(folder_path, model, transform, class_names, output_file):
#     results = []
#     image_paths = list(Path(folder_path).glob('*.png'))


#     for image_path in image_paths:
#         predicted_class, predicted_prob = predict_image(image_path, model, transform, class_names)
#         results.append([image_path.name, predicted_class, predicted_prob])

#     # Save results to a CSV file
#     df = pd.DataFrame(results, columns=['image_name', 'predicted_class', 'probability'])
#     df.to_csv(output_file, index=False)
#     print(f"Predictions saved to {output_file}")

# Predict on all images in a folder
def predict_folder(folder_path, model, transform, class_names, output_file, class_mapping):
    results = []
    image_paths = list(Path(folder_path).glob('*.png'))

    for image_path in image_paths:
        predicted_class, predicted_prob = predict_image(image_path, model, transform, class_names)
        blob_sizes = measure_blob_sizes(image_path)
        
        # Get the min and max pixel values for the predicted class
        min_pixels = class_mapping[predicted_class]['min_pixels']
        max_pixels = class_mapping[predicted_class]['max_pixels']
        
        # Check if any blob size is within the min and max range
        valid_blobs = [size for size in blob_sizes if min_pixels <= size <= max_pixels]
        is_valid = len(valid_blobs) > 0

        results.append([image_path.name, predicted_class, predicted_prob, is_valid])

    # Save results to a CSV file
    df = pd.DataFrame(results, columns=['image_name', 'predicted_class', 'probability', 'valid_blob'])
    df.to_csv(output_file, index=False)
    print(f"Predictions saved to {output_file}")




In [49]:
from pathlib import Path
import json

# Define the base paths using the Path constructor
MODEL_DIR = Path("C:/Data/Python/IFCBclassify_MN/data/models/model_20240706_023737")
IMAGE_FOLDER = Path("C:/Data/Python/IFCB/data/D20240628/")
OUTPUT_FILE = MODEL_DIR / "predictions_with_blobs.csv"

# Model file path
MODEL_PATH = MODEL_DIR / "model.pth"

# Path to class mapping file
class_mapping_path = MODEL_DIR / "class_mapping.json"

# Load the existing class mapping
with open(class_mapping_path, 'r') as f:
    class_mapping = json.load(f)

# Extract class names
class_names = list(class_mapping.keys())

# Load the model
model = load_model(MODEL_PATH, class_names)

# Predict on all images in the folder and save results
predict_folder(IMAGE_FOLDER, model, simple_transform, class_names, OUTPUT_FILE, class_mapping)

# # Define the base paths using the Path constructor

# MODEL_DIR = Path("C:/Data/Python/IFCBclassify_MN/data/models/model_20240706_023737")
# IMAGE_FOLDER = Path("C:/Data/Python/IFCB/data/D20240628/")
# OUTPUT_FILE = MODEL_DIR / "predictions2.csv"

# # Model file path
# MODEL_PATH = MODEL_DIR / "model.pth"

# # Path to class mapping file
# class_mapping_path = MODEL_DIR / "class_mapping.json"

# # Load the existing class mapping
# with open(class_mapping_path, 'r') as f:
#     class_mapping = json.load(f)

# # Extract class names
# class_names = list(class_mapping.keys())

# # for class_name, attributes in class_mapping.items():
# #     print(f"Class Name: {class_name}")
# #     print(f" - ID: {attributes['id']}")
# #     print(f" - Min Pixels: {attributes['min_pixels']}")
# #     print(f" - Max Pixels: {attributes['max_pixels']}")
# #     print("\n")


# #Load the model
# model = load_model(MODEL_PATH, class_names)

# #Predict on all images in the folder and save results
# predict_folder(IMAGE_FOLDER, model, simple_transform, class_names, OUTPUT_FILE)

Predictions saved to C:\Data\Python\IFCBclassify_MN\data\models\model_20240706_023737\predictions_with_blobs.csv


In [50]:
import json
from pathlib import Path

try:
    with open(class_mapping_path, 'r') as f:
        class_mapping = json.load(f)

    # Iterate through each class and update types
    for class_name, attributes in class_mapping.items():
        # Safely convert ID and pixel values to integers
        attributes['id'] = int(attributes.get('id', 0))
        attributes['min_pixels'] = int(attributes.get('min_pixels', 0))
        attributes['max_pixels'] = int(attributes.get('max_pixels', 0))

except json.JSONDecodeError:
    print("Error decoding JSON from file")
except FileNotFoundError:
    print("The class mapping file was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

# Optionally print to verify
for class_name, attributes in class_mapping.items():
    print(f"Class Name: {class_name}, ID: {attributes['id']}, Min Pixels: {attributes['min_pixels']}, Max Pixels: {attributes['max_pixels']}")



Class Name: Achnathes_like_129, ID: 0, Min Pixels: 0, Max Pixels: 36121
Class Name: Air_bubbles_149, ID: 1, Min Pixels: 0, Max Pixels: 256004
Class Name: Akashiwo_sanguinea_057, ID: 2, Min Pixels: 0, Max Pixels: 63028
Class Name: Alexandrium_pseudogonyaulax_060, ID: 3, Min Pixels: 0, Max Pixels: 25462
Class Name: Amphidinium_like_015, ID: 4, Min Pixels: 0, Max Pixels: 9586
Class Name: Apedinella_radians_031, ID: 5, Min Pixels: 0, Max Pixels: 6127
Class Name: Asterionellopsis_glacialis_034, ID: 6, Min Pixels: 0, Max Pixels: 266548
Class Name: Beads_148, ID: 7, Min Pixels: 8, Max Pixels: 4471
Class Name: Centrales_sideview, ID: 8, Min Pixels: 0, Max Pixels: 43447
Class Name: Centrales_topview, ID: 9, Min Pixels: 0, Max Pixels: 86479
Class Name: Cerataulina_pelagica_036, ID: 10, Min Pixels: 0, Max Pixels: 132141
Class Name: Chaetoceroa_affinis_037, ID: 11, Min Pixels: 0, Max Pixels: 373059
Class Name: Chaetoceros_contortus_128, ID: 12, Min Pixels: 0, Max Pixels: 231632
Class Name: Chaetoc